In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [57]:
import dill as pickle

with open("document_samples.pkl", "rb") as f:
    document_samples = pickle.load(f)

with open("sample_labels.pkl", "rb") as f:
    sample_labels = pickle.load(f)

In [22]:
document_samples = [ds.encode('utf-8') for ds in document_samples]

In [40]:
dataset = tf.data.Dataset.from_tensor_slices((tf.constant(document_samples), tf.constant(sample_labels)) )

In [34]:
for element in dataset:
  print(element)
  break


(<tf.Tensor: shape=(), dtype=string, numpy=b'Showers continued throughout the week in\nthe Bahia cocoa zone, alleviating the drought since early\nJanuary and improving prospects for the coming temporao,\nalthough normal humidity levels have not been restored,\nComissaria Smith said in its weekly review.\n    The dry period means the temporao will be late this year.\n    Arrivals for the week ended February 22 were 155,221 bags\nof 60 kilos making a cumulative total for the season of 5.93\nmln against 5.81 at the same stage last year. Again it seems\nthat cocoa delivered earlier on consignment was included in the\narrivals figures.\n    Comissaria Smith said there is still some doubt as to how\nmuch old crop cocoa is still available as harvesting has\npractically come to an end. With total Bahia crop estimates\naround 6.4 mln bags and sales standing at almost 6.2 mln there\nare a few hundred thousand bags still in the hands of farmers,\nmiddlemen, exporters and processors.\n    There ar

In [76]:
y = ["{}.{}".format(sl[0],sl[1]) for sl in sample_labels]

In [67]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [87]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(document_samples)

In [88]:
word_index = tokenizer.word_index

In [89]:
sequences = tokenizer.texts_to_sequences(document_samples)

In [90]:
padded_sequences = pad_sequences(sequences, padding="post" )

In [74]:
max_length = len(padded_sequences[0])

In [96]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_index.keys()), 64, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(len(set(y)), activation="softmax")
    ])

In [102]:
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [109]:
label_map = {k : v for v,k in enumerate(list(set(y)))}

In [111]:
y_ = [label_map[k] for k in y]

In [112]:
history = model.fit(x=padded_sequences, y = np.array(y_), epochs = 30, validation_split=0.2)

Epoch 1/30
358/477 [=====================>........] - ETA: 11:57 - loss: 2.3231 - accuracy: 0.4689

KeyboardInterrupt: 